In [3]:
# Standard library imports
from datetime import datetime
import os

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Visualize loop progress

# Set pandas display options for better data frame visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Segreti
from dotenv import load_dotenv
load_dotenv()

# API
import requests
import json

| Gap-Up Screener |
|--|
TradingView Data (import)

In [4]:
# today's date string for file naming automation
today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
# today_date_str = '_2024-03-07.csv'

print(today_date_str)

_2024-03-08.csv


In [5]:
# concat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
trading_view_raw_df.head(5)

1647 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower"
0,NVDA,NVIDIA Corporation,NASDAQ,2415075006363.128,USD,966.030,USD,954.99,USD,24.00,USD,2.589863,3.053880,2399495000,20958408,253647929,1858893,48115859.1,52323265.09999998,45852531.44444457,4.430353,3.391630,3.630145,960.280000,80.9307586,Electronic Technology,1.37240078,1.8023118,2.3419523,1.7245569,0.528035,0.163001,0.760658,1.690912,2.343450,1.384451,0.65999561,0.38587557,1.13352072,0.29964513,968.61,USD,968.61,USD,968.61,USD,968.61,USD,968.610000,USD,967.50,USD,968.08,USD,968.6100,USD,968.610,USD,968.610,USD,968.610,USD,968.610,USD,968.61,USD,968.61,USD,2058175.67567568,USD,965.82330,965.764850,965.85680,957.694200,967.871452,965.727665,963.583878,982.643420,945.508915,908.374410
1,META,"Meta Platforms, Inc.",NASDAQ,1326710509490.8638,USD,520.400,USD,514.05,USD,2.02,USD,0.394385,0.363146,2193265045.7999997,4190043,62183845,144768,13142696,20500910.6,17812733.1,2.220055,2.153452,2.087621,518.856667,34.93484959,Technology Services,1.38229501,1.301791,1.1534996,1.2083921,0.482799,0.166259,0.825242,1.382486,2.087941,1.386662,0.67897117,0.28775554,0.70743387,0.17687798,523.57,USD,523.57,USD,523.57,USD,523.57,USD,523.570000,USD,520.98,USD,523.45,USD,523.5700,USD,523.570,USD,523.570,USD,523.570,USD,523.570,USD,523.57,USD,523.57,USD,2003981.16374764,USD,521.05882,521.499125,521.73402,519.031100,523.680272,522.087405,520.494538,524.796416,516.514460,508.232504
2,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,NYSE,626312399470.3125,USD,157.870,USD,153.29,USD,4.70,USD,3.150134,2.741287,5184836336.448,14347642,111555365,863836,17021464.49999999,14822432.66666667,12086126.76666667,5.909595,3.636336,2.961051,156.480000,30.46271998,Electronic Technology,3.32223718,1.1705236,1.2532234,1.1473486,0.298988,0.146521,0.984316,1.292134,3.661480,2.429691,1.23989584,0.86436604,1.79941585,0.60763541,158.26,USD,158.26,USD,158.26,USD,158.26,USD,158.260000,USD,157.98,USD,158.26,USD,158.2600,USD,158.260,USD,158.260,USD,158.260,USD,158.260,USD,158.26,USD,158.26,USD,-,-,157.53300,157.558125,157.50002,156.230392,157.984276,157.401430,156.818584,160.689949,154.244255,147.798561
3,TSLA,"Tesla, Inc.",NASDAQ,570045637014.3033,USD,178.990,USD,179.48,USD,2.85,US

In [6]:
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans
        (df['Market capitalization'] >= 200000000000),  # 200 billion and above
        #Large Cap
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),  # 10 billion to 200 billion
        # Midlers
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),  # 2 billion to 10 billion
        # Small Cap
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000),  # 300 million to 2 billion
        # Micro Cap
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),  # 300 million and below (excluding 50 million and below)
        # Shrimp
        (df['Market capitalization'] <= 50000000)  # 50 million and below
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df


In [7]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,NVDA,NVIDIA Corporation,NASDAQ,2.415075e+12,USD,966.030,USD,954.99,USD,24.00,USD,2.589863,3.053880,2399495000,20958408,253647929,1858893,48115859.1,52323265.09999998,45852531.44444457,4.430353,3.391630,3.630145,960.280000,80.9307586,Electronic Technology,1.37240078,1.8023118,2.3419523,1.7245569,0.528035,0.163001,0.760658,1.690912,2.343450,1.384451,0.65999561,0.38587557,1.13352072,0.29964513,968.61,USD,968.61,USD,968.61,USD,968.61,USD,968.610000,USD,967.50,USD,968.08,USD,968.6100,USD,968.610,USD,968.610,USD,968.610,USD,968.610,USD,968.61,USD,968.61,USD,2058175.67567568,USD,965.82330,965.764850,965.85680,957.694200,967.871452,965.727665,963.583878,982.643420,945.508915,908.374410,Titans
1,META,"Meta Platforms, Inc.",NASDAQ,1.326711e+12,USD,520.400,USD,514.05,USD,2.02,USD,0.394385,0.363146,2193265045.7999997,4190043,62183845,144768,13142696,20500910.6,17812733.1,2.220055,2.153452,2.087621,518.856667,34.93484959,Technology Services,1.38229501,1.301791,1.1534996,1.2083921,0.482799,0.166259,0.825242,1.382486,2.087941,1.386662,0.67897117,0.28775554,0.70743387,0.17687798,523.57,USD,523.57,USD,523.57,USD,523.57,USD,523.570000,USD,520.98,USD,523.45,USD,523.5700,USD,523.570,USD,523.570,USD,523.570,USD,523.570,USD,523.57,USD,523.57,USD,2003981.16374764,USD,521.05882,521.499125,521.73402,519.031100,523.680272,522.087405,520.494538,524.796416,516.514460,508.232504,Titans
2,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,NYSE,6.263124e+11,USD,157.870,USD,153.29,USD,4.70,USD,3.150134,2.741287,5184836336.448,14347642,111555365,863836,17021464.49999999,14822432.66666667,12086126.76666667,5.909595,3.636336,2.961051,156.480000,30.46271998,Electronic Technology,3.32223718,1.1705236,1.2532234,1.1473486,0.298988,0.146521,0.984316,1.292134,3.661480,2.429691,1.23989584,0.86436604,1.79941585,0.60763541,158.26,USD,158.26,USD,158.26,USD,158.26,USD,158.260000,USD,157.98,USD,158.26,USD,158.2600,USD,158.260,USD,158.260,USD,158.260,USD,158.260,USD,158.26,USD,158.26,USD,-,-,157.53300,157.558125,157.50002,156.230392,157.984276,157.401430,156.818584,160.689949,154.244255,147.798561,Titans
3,TSLA,"Tesla, Inc.",NASDAQ,5.700456e+11,USD,178.990,USD,179.

In [8]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code dumb dumb!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [9]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 1647 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Small caps,399,24.225865
1,Midlers,386,23.436551
2,Large caps,335,20.340012
3,Shrimp,258,15.664845
4,Micro caps,233,14.146934
5,Titans,21,1.275046
6,Undefined,15,0.910747


In [10]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(5)

1632 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
1219,FPL,First Trust New Opportunities MLP & Energy Fund,NYSE,1.789650e+08,USD,7.2800,USD,7.75,USD,0.49,USD,6.749311,6.749311,24650676.746,38941,437036,100,82363.6,81261.06666667,60656.18888889,0.413223,1.205189,1.434731,7.276667,12.74509804,Miscellaneous,-,0.6614087,0.7047051,1.2954613,0.029587,1.058369,3.267559,7.328563,2.467977,1.198512,0.6918458,0.46334394,1.39406757,0.50789424,7.29,USD,7.2900,USD,7.29,USD,7.29,USD,20.75,USD,7.280,USD,7.28,USD,7.290,USD,7.29,USD,7.29,USD,7.29,USD,7.29,USD,7.29,USD,7.29,USD,-,-,7.27140,7.272125,7.27940,7.269638,7.295533,7.269285,7.243037,7.299704,7.262130,7.224556,Micro caps
387,AR,Antero Resources Corporation,NYSE,8.065801e+09,USD,26.5700,USD,26.45,USD,0.02,USD,0.075529,-0.113293,279870571.21599996,710020,17367566,4800,3649572.6,5147041.96666667,5068956.08888889,1.628171,2.259694,3.407306,26.606667,34.44833398,Energy Minerals,1.06783022,1.2415539,0.6095499,3.31293,0.043446,0.020196,0.136824,0.676755,1.814102,1.045500,0.39810884,0.17163704,0.71309573,0.19390449,27.30,USD,27.3000,USD,30.90,USD,30.90,USD,68.43,USD,26.590,USD,26.70,USD,26.800,USD,26.84,USD,26.84,USD,26.84,USD,26.84,USD,27.30,USD,27.30,USD,7476238.41059603,USD,26.58500,26.590625,26.61400,26.652262,26.711419,26.632515,26.553611,26.790372,26.601970,26.413568,Midlers
909,PCT,"PureCycle Technologies, Inc.",NASDAQ,9.367067e+08,USD,5.7000,USD,5.20,USD,0.08,USD,1.568627,1.960784,125854316.4,908434,17865017,10683,2698892.1,2777844.06666667,3785081.05555556,13.435701,16.350100,11.015054,5.570000,-,Industrial Services,1.71841387,2.0132399,1.543707,1.4254513,0.603768,0.113715,0.381025,1.047285,1.536605,0.742183,0.51059475,0.32471021,1.3210073,0.34312114,6.50,USD,6.5000,USD,8.81,USD,11.89,USD,35.75,USD,5.716,USD,5.77,USD,5.800,USD,5.80,USD,5.80,USD,5.80,USD,5.80,USD,6.50,USD,6.50,USD,0,USD,5.70448,5.697662,5.69840,5.611246,5.764167,5.712525,5.660883,6.027858,5.411095,4.794332,Small caps
31,PFE,"Pfizer, Inc.",NYSE,1.524094e+11,USD,26.9905,USD,26.90,USD,0.13,USD,0.485256,0.410601,5644436586.3,6920751,298687845,254849,51501633.3,46297727.89999998,43799607.75555556,1.303538,2.251211,1.906126,26.993500,75.09877574,Health Technology,0.6265

In [11]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

market_cap_type_df2 = tv_df['marketCapType'].value_counts().reset_index()
market_cap_type_df2.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df2['percentage'] = (market_cap_type_df2['count'] / market_cap_type_df2['count'].sum()) * 100

market_cap_type_df2
print(market_cap_type_df2)


  marketCapType  count  percentage
0    Small caps    399   24.448529
1       Midlers    386   23.651961
2    Large caps    335   20.526961
3        Shrimp    258   15.808824
4    Micro caps    233   14.276961
5        Titans     21    1.286765


In [12]:
# 1/3 #### CONFIG DATA SECTION ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # Reduced to 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # Increased to 1 billion shares
        "relative_volume_threshold": 1.2,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # Reduced to 0.1%
        "pre_market_vmap_drawdown_threshold": 0.003  # .3% drawdown from VWAP
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # Reduced to 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # Increased to 200 million shares
        "relative_volume_threshold": 1.3,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # Reduced to 0.5%
        "pre_market_vmap_drawdown_threshold": 0.004  # .4% drawdown from VWAP
    },
    "Midlers": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .02,
        "pre_market_vmap_drawdown_threshold": 0.005  # .5% drawdown from VWAP
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .03,
        "pre_market_vmap_drawdown_threshold": 0.006  # 6% drawdown from VWAP
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .04,
        "pre_market_vmap_drawdown_threshold": 0.007  # 7% drawdown from VWAP
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .05,
        "pre_market_vmap_drawdown_threshold": 0.008  # 8% drawdown from VWAP
    }
}

# 2/3 #### CORE FUNCTIONS ####
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    # create a copy to safely modify without affecting the original DataFrame
    modified_df = df.copy()
    modified_df['Float shares outstanding'] = pd.to_numeric(modified_df['Float shares outstanding'], errors='coerce')
    return modified_df[modified_df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df['Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    # Convert 'Relative Volume at Time' to float
    df['Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vmap_drawdown_threshold):
    """
    Filter stocks where the price is near the VWAP, considering the specified drawdown threshold.
    """
    df_filtered = df.copy()
    df_filtered['Price'] = pd.to_numeric(df_filtered['Price'], errors='coerce')
    df_filtered['Volume Weighted Average Price 1 day'] = pd.to_numeric(df_filtered['Volume Weighted Average Price 1 day'], errors='coerce')
    
    # Calculate the acceptable minimum price based on the VWAP drawdown threshold
    df_filtered['Min Price from VWAP'] = df_filtered['Volume Weighted Average Price 1 day'] * (1 - pre_market_vmap_drawdown_threshold)
    
    # Apply the filter
    df_filtered = df_filtered[df_filtered['Price'] >= df_filtered['Min Price from VWAP']]
    
    return df_filtered.drop(columns=['Min Price from VWAP'])

"""SPECIAL FILTERS"""
def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    # Ensure data types are correct; convert to numeric if necessary
    df['Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df['Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df['Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    filtered_df = df[
        (df['Volatility 1 day'] >= df['Volatility 1 week']) & 
        (df['Volatility 1 day'] >= df['Volatility 1 month'])]
    return filtered_df

# 3/3 #### WRAPPER FUNCTION ####
def screen_stocks_by_category(df, category):
    """Screen stocks based on their market cap category."""
    # 3a. Get the configuration for the specified category
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vmap_drawdown_threshold = config.get("pre_market_vmap_drawdown_threshold", 0)
    # 3b. Apply the filters
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df) # SPECIAL FILTER
    # filtered_df = filter_by_price_above_vwap(filtered_df) # SPECIAL FILTER
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vmap_drawdown_threshold)
  
    # 3c. Return the filtered DataFrame
    return filtered_df

In [13]:
#EXECUTE
tv_L2_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    screened_df = screen_stocks_by_category(category_df, category)
    tv_L2_df = pd.concat([tv_L2_df, screened_df])

# Display results
print(f'Original DataFrame length: {len(tv_df.index)}')
print(f'Number of rows removed: {len(tv_df.index) - len(tv_L2_df.index)}')
print(f'NEW DataFrame length: {len(tv_L2_df.index)}')

# filter in descending order by market cap and then by pre-market change percentage
tv_L2_df = tv_L2_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# reset index and drop
tv_L2_df = tv_L2_df.reset_index(drop=True)

Original DataFrame length: 1632
Number of rows removed: 1628
NEW DataFrame length: 4


In [14]:
# create a breakdown of the market cap types
market_cap_type_df3 = tv_L2_df['marketCapType'].value_counts().reset_index()
market_cap_type_df3.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df3['percentage'] = (market_cap_type_df3['count'] / market_cap_type_df3['count'].sum()) * 100

print('Returned Gap-Up Market Cap Type Groups:')
market_cap_type_df3

Returned Gap-Up Market Cap Type Groups:


,marketCapType,count,percentage
0,Small caps,2,50.0
1,Large caps,1,25.0
2,Shrimp,1,25.0


In [15]:
# market cap type groups
print(f"Market Cap Groups:\n{tv_L2_df['marketCapType'].unique()}\n")

# TradingView column names
tv_raw_columns_list = tv_L2_df.columns.tolist()
print(f'TradingView Column Names (raw data):\n{tv_raw_columns_list}\n')

# stocks returned from the pre-market gap up screener
print(f'{len(tv_L2_df.index)} = U.S. stocks returned from the pre-market gap up screener.')

# create a list of 'Symbols' from the tv_L2_df
gap_up_stocks_list = tv_L2_df['Symbol']
gap_up_stocks_list = gap_up_stocks_list.tolist()
print(gap_up_stocks_list)

# FILTERD DATAFRAME WITH GAP-UP STOCK TARGETS
tv_L2_df


Market Cap Groups:
['Large caps' 'Small caps' 'Shrimp']

TradingView Column Names (raw data):
['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Pre-market Open', 'Pre-market Open - Currency', 'Pre-market Change', 'Pre-market Change - Currency', 'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding', 'Volume 1 day', 'Volume 1 week', 'Pre-market Volume', 'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Relative Volume at Time', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years', 'Relative Volume 1 minute', 'Relative Volume 5 minutes', 'Relative Volume 15 minutes', 'Relative Volume 30 minutes', 'Relative Volume 1 hour', 'Relative Volume 2 hours', 'Relative Volume 4 hours', 'Relative Volume 1 day', 'Relative Volume 1 week', 'Relative Vo

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,IOT,Samsara Inc.,NYSE,2.130625e+10,USD,39.43,USD,39.70,USD,2.62,USD,7.625146,15.541327,1.670437e+08,6886538,29738714,233217,5662199,3820710.16666667,3491359.37777778,15.206612,6.326014,5.004297,38.536667,-,Technology Services,8.704419,2.327024,1.5505408,1.5505408,0.891138,0.228782,0.636827,2.093271,10.253309,6.742558,2.88581651,1.323699,2.09731349,0.50966,39.88,USD,39.880,USD,39.88,USD,39.880,USD,39.88,USD,39.6383,USD,39.6383,USD,39.670,USD,39.88,USD,39.88,USD,39.88,USD,39.88,USD,39.88,USD,39.88,USD,-,-,39.53704,39.479400,39.34700,38.777046,39.663237,39.36636,39.069483,42.133411,37.099940,32.066469,Large caps
1,HCI,"HCI Group, Inc.",NYSE,1.070808e+09,USD,111.68,USD,106.15,USD,13.79,USD,13.899808,6.995263,6.996457e+06,174737,583229,5106,84991.6,92108.6,124080.56666667,14.619003,3.339954,2.830011,111.563367,-,Finance,20.897273,0.30785272,1.4800545,1.1199094,0.047079,0.199941,1.754771,3.471104,20.879901,12.395502,5.98022859,2.375435,1.29830012,0.34135341,115.00,USD,115.000,USD,115.00,USD,115.000,USD,139.80,USD,112.3900,USD,113.3050,USD,113.305,USD,115.00,USD,115.00,USD,115.00,USD,115.00,USD,115.00,USD,115.00,USD,-,-,111.95326,112.003912,111.54826,110.065346,113.087486,111.34383,109.600174,118.062834,106.198420,94.334006,Small caps
2,CRMT,"America's Car-Mart, Inc.",NASDAQ,4.539554e+08,USD,71.01,USD,63.13,USD,0.66,USD,1.059390,1.332263,5.738424e+06,169192,1019174,10076,142439,104218.33333333,104139.47777778,18.266254,7.063429,5.197211,69.903333,-,Retail Trade,6.997781,2.5008163,1.4059496,1.6846163,0.100110,0.059508,0.522739,1.499686,7.636282,4.678565,2.30682163,1.228084,2.66671795,0.64590469,74.10,USD,83.070,USD,95.99,USD,127.955,USD,177.45,USD,71.0100,USD,72.0700,USD,73.740,USD,74.10,USD,74.10,USD,74.10,USD,74.10,USD,74.10,USD,74.10,USD,621901.7699115,USD,70.96100,71.072500,71.38500,71.023077,73.441200,71.84019,70.239180,77.209012,68.144000,59.078988,Small caps
3,PBM,Psyence Biomedical Ltd.,NASDAQ,1.700614e+07,USD,3.97,USD,1.37,USD,1.17,USD,92.125984,7.874016,7.909118e+05,96839409,115213200,20808840,11566350.5,4056244.3,1360370.86666667,135.111111,67.860565,26.029400,3.510000,-,Health Technology,407.985235,-1.0776684,-0.61

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [16]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [17]:
"""NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

info_news_df

,Symbol,Exchange,marketCapType
0,IOT,NYSE,Large caps
1,HCI,NYSE,Small caps
2,CRMT,NASDAQ,Small caps
3,PBM,NASDAQ,Shrimp


In [18]:
# # manual add list
# manual_add_list = ['DELL','PLTR','TSLA']
# # append list with manually added stocks
# gap_up_stocks_list.extend(manual_add_list)

# print(f'{manual_add_list} added manually\n')
# print(f'New list: {gap_up_stocks_list}')

In [19]:
#### IEX CLOUD API KEY -> VARIABLE ####
iex_api_key = os.getenv("IEX_API_KEY")

# set the list of symbols variable
symbols = ','.join(gap_up_stocks_list)

# SETTING THE DATE RANGE
"""
y - Example: range=1y -> returns (one year)
ytd - Example: range=ytd -> returns (year-to-date)
m - Example: range=6m returns (six months)
d - Example: range=5d returns (five days)
"""
data_range = '5m' # DATE RANGE (for historical stock price data)

url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_api_key}'

# Make the GET request
response = requests.get(url)

# Initialize a list to collect DataFrames for each symbol
dataframes_list = []

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Process and print the data with a progress bar
    for symbol in tqdm(gap_up_stocks_list, desc="Processing symbols"):
        if symbol in data:
            # Create a DataFrame for the current symbol's data
            symbol_df = pd.DataFrame(data[symbol]['chart'])
            # Add a 'Ticker' column to identify the data
            symbol_df['Ticker'] = symbol
            # Append the current symbol's DataFrame to the list
            dataframes_list.append(symbol_df)
            # print(f'Data for {symbol}:')
            # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
else:
    print(f'Failed to fetch data: {response.status_code}')

# concatenate dataframes and create a combined list
historical_df = pd.concat(dataframes_list, ignore_index=True)


Processing symbols: 100%|██████████| 4/4 [00:00<00:00, 932.64it/s]


In [20]:
# Set variables for historical data analysis
historical_length = len(historical_df.index)
unique_stocks = historical_df['Ticker'].nunique()
start_date = historical_df['date'].min()
end_date = historical_df['date'].max()

print(f"START date: {start_date}")
print(f"END date: {end_date}")
print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
print(f"{historical_length} = Length of dataframe")
print(f"{unique_stocks} = Unique stocks in dataframe")

# Entries per stock
days_per_stock = historical_length / unique_stocks
print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# View the stocks
print(f'Unique stock ticker symbols: {gap_up_stocks_list}')
historical_cols_list = historical_df.columns.tolist()
print(f'Columns: {historical_cols_list}')

# convert df to .csv
historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# Display the first few rows of the dataframe
historical_df.head()

START date: 2023-10-09
END date: 2024-03-07
104 days of historical stock data.

341 = Length of dataframe
4 = Unique stocks in dataframe
85.25 = Date entries per stock (MATH check -> should be the same as the unique )

Unique stock ticker symbols: ['IOT', 'HCI', 'CRMT', 'PBM']
Columns: ['close', 'high', 'low', 'open', 'priceDate', 'symbol', 'volume', 'id', 'key', 'subkey', 'date', 'updated', 'changeOverTime', 'marketChangeOverTime', 'uOpen', 'uClose', 'uHigh', 'uLow', 'uVolume', 'fOpen', 'fClose', 'fHigh', 'fLow', 'fVolume', 'label', 'change', 'changePercent', 'Ticker']


,close,high,low,open,priceDate,symbol,volume,id,key,subkey,date,updated,changeOverTime,marketChangeOverTime,uOpen,uClose,uHigh,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent,Ticker
0,25.58,25.8180,25.030,25.19,2023-10-09,IOT,1356147,HISTORICAL_PRICES,IOT,,2023-10-09,1696899638000,0.000000,0.000000,25.19,25.58,25.8180,25.030,1356147,25.19,25.58,25.8180,25.030,1356147,"Oct 9, 23",0.00,0.0000,IOT
1,25.04,26.2400,24.920,25.50,2023-10-10,IOT,2846765,HISTORICAL_PRICES,IOT,,2023-10-10,1696986044000,-0.021110,-0.021110,25.50,25.04,26.2400,24.920,2846765,25.50,25.04,26.2400,24.920,2846765,"Oct 10, 23",-0.54,-0.0211,IOT
2,25.43,26.0900,25.070,25.63,2023-10-11,IOT,1795656,HISTORICAL_PRICES,IOT,,2023-10-11,1697072459000,-0.005864,-0.005864,25.63,25.43,26.0900,25.070,1795656,25.63,25.43,26.0900,25.070,1795656,"Oct 11, 23",0.39,0.0156,IOT
3,24.65,25.6700,24.335,25.48,2023-10-12,IOT,1648330,HISTORICAL_PRICES,IOT,,2023-10-12,1697164245000,-0.036357,-0.036357,25.48,24.65,25.6700,24.335,1648330,25.48,24.65,25.6700,24.335,1648330,"Oct 12, 23",-0.78,-0.0307,IOT
4,23.46,24.5187,22.740,24.20,2023-10-13,IOT,2797597,HISTORICAL_PRICES,IOT,,2023-10-13,1697247015000,-0.082877,-0.082877,24.20,23.46,24.5187,22.740,2797597,24.20,23.46,24.5187,22.740,2797597,"Oct 13, 23",-1.19,-0.0483,IOT


In [21]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [22]:
# Ensure the dataframe is sorted by date if it's not already
historical_sorted_df = historical_df.sort_values(by='date')

# Select mandatory columns for a historical price bar chart + volume
# and explicitly copy the dataframe to ensure it's not a view but a new object
chart_historical_df = historical_sorted_df[[
    'symbol',   # *MANDATORY
    'date',     # *MANDATORY
    'close',    # *MANDATORY
    'high',     # *MANDATORY
    'low',      # *MANDATORY
    'open',     # *MANDATORY
    'volume'    # *MANDATORY
]].copy()

# Group by 'symbol' and get the last occurrence
last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

last_date_df


,symbol,date,close,high,low,open,volume
0,CRMT,2024-03-07,62.30,63.735,60.6900,61.9600,194314
1,HCI,2024-03-07,99.21,99.960,97.1050,98.5100,96791
2,IOT,2024-03-07,34.36,34.400,33.0500,34.0000,7593193
3,PBM,2024-03-07,1.27,1.500,0.9401,0.9788,3711232


In [23]:
# re-index the dataframe
chart_historical_df.reset_index(drop=True, inplace=True)
# view head of your main df
print(f'Length of historical data: {len(chart_historical_df.index)}')

Length of historical data: 341


In [24]:
"""
YOUR DATAFRAMES TO MOVE FORWARD WITH:
"""
#### INFO / NEWS DATAFRAME ####
print(f'1 - NEWS & STOCK INFO (trending news etc.):\n{info_news_df}\n')

#### IEX (API) - HISTORICAL DATA FRAME ####
print('2a - IEX HISTORICAL data (START):')
print(f'{chart_historical_df.head()}\n')
print('2b - IEX HISTORICAL data (END):')
print(f'{chart_historical_df.tail()}\n')

1 - NEWS & STOCK INFO (trending news etc.):
  Symbol Exchange marketCapType
0    IOT     NYSE    Large caps
1    HCI     NYSE    Small caps
2   CRMT   NASDAQ    Small caps
3    PBM   NASDAQ        Shrimp

2a - IEX HISTORICAL data (START):
  symbol        date  close    high     low   open   volume
0    IOT  2023-10-09  25.58  25.818  25.030  25.19  1356147
1    HCI  2023-10-09  55.41  55.690  53.785  54.47    47210
2   CRMT  2023-10-09  86.96  87.885  85.955  87.16    78228
3   CRMT  2023-10-10  88.40  90.150  87.130  87.13    52196
4    IOT  2023-10-10  25.04  26.240  24.920  25.50  2846765

2b - IEX HISTORICAL data (END):
    symbol        date  close    high      low     open   volume
336    HCI  2024-03-06  98.16  98.400  97.0250  97.6100   115636
337   CRMT  2024-03-07  62.30  63.735  60.6900  61.9600   194314
338    IOT  2024-03-07  34.36  34.400  33.0500  34.0000  7593193
339    HCI  2024-03-07  99.21  99.960  97.1050  98.5100    96791
340    PBM  2024-03-07   1.27   1.500   0.9

In [25]:
#### TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY'S DATA) ####
print('3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):')
print(f'U.S. stocks returned from the pre-market gap up screener = {len(tv_L2_df.index)}')
tv_L2_df

3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):
U.S. stocks returned from the pre-market gap up screener = 4


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,IOT,Samsara Inc.,NYSE,2.130625e+10,USD,39.43,USD,39.70,USD,2.62,USD,7.625146,15.541327,1.670437e+08,6886538,29738714,233217,5662199,3820710.16666667,3491359.37777778,15.206612,6.326014,5.004297,38.536667,-,Technology Services,8.704419,2.327024,1.5505408,1.5505408,0.891138,0.228782,0.636827,2.093271,10.253309,6.742558,2.88581651,1.323699,2.09731349,0.50966,39.88,USD,39.880,USD,39.88,USD,39.880,USD,39.88,USD,39.6383,USD,39.6383,USD,39.670,USD,39.88,USD,39.88,USD,39.88,USD,39.88,USD,39.88,USD,39.88,USD,-,-,39.53704,39.479400,39.34700,38.777046,39.663237,39.36636,39.069483,42.133411,37.099940,32.066469,Large caps
1,HCI,"HCI Group, Inc.",NYSE,1.070808e+09,USD,111.68,USD,106.15,USD,13.79,USD,13.899808,6.995263,6.996457e+06,174737,583229,5106,84991.6,92108.6,124080.56666667,14.619003,3.339954,2.830011,111.563367,-,Finance,20.897273,0.30785272,1.4800545,1.1199094,0.047079,0.199941,1.754771,3.471104,20.879901,12.395502,5.98022859,2.375435,1.29830012,0.34135341,115.00,USD,115.000,USD,115.00,USD,115.000,USD,139.80,USD,112.3900,USD,113.3050,USD,113.305,USD,115.00,USD,115.00,USD,115.00,USD,115.00,USD,115.00,USD,115.00,USD,-,-,111.95326,112.003912,111.54826,110.065346,113.087486,111.34383,109.600174,118.062834,106.198420,94.334006,Small caps
2,CRMT,"America's Car-Mart, Inc.",NASDAQ,4.539554e+08,USD,71.01,USD,63.13,USD,0.66,USD,1.059390,1.332263,5.738424e+06,169192,1019174,10076,142439,104218.33333333,104139.47777778,18.266254,7.063429,5.197211,69.903333,-,Retail Trade,6.997781,2.5008163,1.4059496,1.6846163,0.100110,0.059508,0.522739,1.499686,7.636282,4.678565,2.30682163,1.228084,2.66671795,0.64590469,74.10,USD,83.070,USD,95.99,USD,127.955,USD,177.45,USD,71.0100,USD,72.0700,USD,73.740,USD,74.10,USD,74.10,USD,74.10,USD,74.10,USD,74.10,USD,74.10,USD,621901.7699115,USD,70.96100,71.072500,71.38500,71.023077,73.441200,71.84019,70.239180,77.209012,68.144000,59.078988,Small caps
3,PBM,Psyence Biomedical Ltd.,NASDAQ,1.700614e+07,USD,3.97,USD,1.37,USD,1.17,USD,92.125984,7.874016,7.909118e+05,96839409,115213200,20808840,11566350.5,4056244.3,1360370.86666667,135.111111,67.860565,26.029400,3.510000,-,Health Technology,407.985235,-1.0776684,-0.61

In [26]:
"""PICK UP HERE WHERE YOU LEFT OFF"""
# you need to get the chart dataframe to a place where it has all the data you need in there. 
# it would be more next level if we let it rip into live stock feeds from here. all setup with the correct lines.
# in the charts live feed we could use the tradingview html widget to display the charts.


'PICK UP HERE WHERE YOU LEFT OFF'

| KEY LEVELS (Support & Resistance) |
|-|
| TECHNICAL INDICATOR CALCULATIONS FROM HISTORICAL DATA |

in order of priority of in scope. 
thinking of moving some of the levels out as i test them and see what they look like.
for instance i am thinking of eventually completely replacing MAs with EMAs

In [27]:
"calculate key levels from historical data"
# save historical data to a csv
chart_historical_df.to_csv(f'chart_historical_gpt_data{today_date_str}.csv', index=False)

chart_historical_df.head()

,symbol,date,close,high,low,open,volume
0,IOT,2023-10-09,25.58,25.818,25.030,25.19,1356147
1,HCI,2023-10-09,55.41,55.690,53.785,54.47,47210
2,CRMT,2023-10-09,86.96,87.885,85.955,87.16,78228
3,CRMT,2023-10-10,88.40,90.150,87.130,87.13,52196
4,IOT,2023-10-10,25.04,26.240,24.920,25.50,2846765


In [28]:
"""
IMPORTANT:
for effective plotting and time series analysis, it's crucial that the date column in the df is in the datetime format rather than an object (string). 
"""
chart_historical_df['date'] = pd.to_datetime(chart_historical_df['date'])
chart_historical_df.set_index('date', inplace=True)

# view the multi-indexed dataframe
chart_historical_df.head()

,symbol,close,high,low,open,volume
date,,,,,,
2023-10-09,IOT,25.58,25.818,25.030,25.19,1356147
2023-10-09,HCI,55.41,55.690,53.785,54.47,47210
2023-10-09,CRMT,86.96,87.885,85.955,87.16,78228
2023-10-10,CRMT,88.40,90.150,87.130,87.13,52196
2023-10-10,IOT,25.04,26.240,24.920,25.50,2846765


In [29]:
"""SIMPLE MOVING AVERAGES (SMA)"""
def calculate_sma(dataframe, period):
    return dataframe['close'].rolling(window=period).mean()

chart_historical_df['SMA5'] = calculate_sma(chart_historical_df, 5)
chart_historical_df['SMA10'] = calculate_sma(chart_historical_df, 10)
chart_historical_df['SMA15'] = calculate_sma(chart_historical_df, 15)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15
date,,,,,,,,,
2024-03-05,CRMT,62.55,64.570,62.4600,63.5200,160505,26.5461,49.81105,46.582833
2024-03-05,HCI,97.08,100.050,96.8200,98.7200,92858,38.7821,56.02005,46.546833
2024-03-06,IOT,33.35,34.300,33.1600,33.6100,3810553,45.3280,49.41205,48.733500
2024-03-06,PBM,1.02,1.380,0.7500,0.7900,6566906,38.9720,42.74505,44.312167
2024-03-06,CRMT,60.75,63.130,59.2900,61.0000,211956,50.9500,42.40205,46.058833
2024-03-06,HCI,98.16,98.400,97.0250,97.6100,115636,58.0720,42.30905,52.564700
2024-03-07,CRMT,62.30,63.735,60.6900,61.9600,194314,51.1160,44.94905,54.385367
2024-03-07,IOT,34.36,34.400,33.0500,34.0000,7593193,51.3180,48.32300,50.047367
2024-03-07,HCI,99.21,99.960,97.1050,98.5100,96791,70.9560,54.96400,52.148700


In [30]:
"""EXPONENTIAL MOVING AVERAGES (EMA)"""
def calculate_ema(dataframe, period):
    return dataframe['close'].ewm(span=period, adjust=False).mean()

# calculate EMAs for the specified periods
chart_historical_df['EMA5'] = calculate_ema(chart_historical_df, 5)
chart_historical_df['EMA10'] = calculate_ema(chart_historical_df, 10)
chart_historical_df['EMA20'] = calculate_ema(chart_historical_df, 20)
chart_historical_df['EMA50'] = calculate_ema(chart_historical_df, 50)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50
date,,,,,,,,,,,,,
2024-03-05,CRMT,62.55,64.570,62.4600,63.5200,160505,26.5461,49.81105,46.582833,37.995806,41.506198,44.914993,47.436577
2024-03-05,HCI,97.08,100.050,96.8200,98.7200,92858,38.7821,56.02005,46.546833,57.690537,51.610526,49.883089,49.383378
2024-03-06,IOT,33.35,34.300,33.1600,33.6100,3810553,45.3280,49.41205,48.733500,49.577025,48.290430,48.308509,48.754618
2024-03-06,PBM,1.02,1.380,0.7500,0.7900,6566906,38.9720,42.74505,44.312167,33.391350,39.695807,43.804841,46.882672
2024-03-06,CRMT,60.75,63.130,59.2900,61.0000,211956,50.9500,42.40205,46.058833,42.510900,43.523842,45.418666,47.426489
2024-03-06,HCI,98.16,98.400,97.0250,97.6100,115636,58.0720,42.30905,52.564700,61.060600,53.457689,50.441650,49.416039
2024-03-07,CRMT,62.30,63.735,60.6900,61.9600,194314,51.1160,44.94905,54.385367,61.473733,55.065382,51.571017,49.921292
2024-03-07,IOT,34.36,34.400,33.0500,34.0000,7593193,51.3180,48.32300,50.047367,52.435822,51.300767,49.931872,49.311045
2024-03-07,HCI,99.21,99.960,97.1050,98.5100,96791,70.9560,54.96400,52.148700,68.027215,60.011536,54.625027,51.267867


In [31]:
"""MOVING AVERAGE CONVERGENCE DIVERGENCE (MACD)"""
# calculate the MACD line and the Signal line
chart_historical_df['MACD'] = calculate_ema(chart_historical_df, 12) - calculate_ema(chart_historical_df, 26)
chart_historical_df['MACD_Signal'] = chart_historical_df['MACD'].ewm(span=9, adjust=False).mean()

# MACD Histogram -> useful for visualizing momentum and potential buy/sell signals
chart_historical_df['MACD_Histogram'] = chart_historical_df['MACD'] - chart_historical_df['MACD_Signal']

chart_historical_df.tail(10)

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram
date,,,,,,,,,,,,,,,,
2024-03-05,CRMT,62.55,64.570,62.4600,63.5200,160505,26.5461,49.81105,46.582833,37.995806,41.506198,44.914993,47.436577,-3.278784,-1.185675,-2.093109
2024-03-05,HCI,97.08,100.050,96.8200,98.7200,92858,38.7821,56.02005,46.546833,57.690537,51.610526,49.883089,49.383378,1.314903,-0.685559,2.000462
2024-03-06,IOT,33.35,34.300,33.1600,33.6100,3810553,45.3280,49.41205,48.733500,49.577025,48.290430,48.308509,48.754618,-0.184915,-0.585431,0.400516
2024-03-06,PBM,1.02,1.380,0.7500,0.7900,6566906,38.9720,42.74505,44.312167,33.391350,39.695807,43.804841,46.882672,-3.936909,-1.255726,-2.681183
2024-03-06,CRMT,60.75,63.130,59.2900,61.0000,211956,50.9500,42.40205,46.058833,42.510900,43.523842,45.418666,47.426489,-2.066855,-1.417952,-0.648903
2024-03-06,HCI,98.16,98.400,97.0250,97.6100,115636,58.0720,42.30905,52.564700,61.060600,53.457689,50.441650,49.416039,2.406115,-0.653139,3.059253
2024-03-07,CRMT,62.30,63.735,60.6900,61.9600,194314,51.1160,44.94905,54.385367,61.473733,55.065382,51.571017,49.921292,3.022531,0.081995,2.940536
2024-03-07,IOT,34.36,34.400,33.0500,34.0000,7593193,51.3180,48.32300,50.047367,52.435822,51.300767,49.931872,49.311045,1.242202,0.314037,0.928165
2024-03-07,HCI,99.21,99.960,97.1050,98.5100,96791,70.9560,54.96400,52.148700,68.027215,60.011536,54.625027,51.267867,5.006421,1.252513,3.753907


In [32]:
#### DEV - DATA ORGANIZATION ####
chart_historical_df.columns

Index(['symbol', 'close', 'high', 'low', 'open', 'volume', 'SMA5', 'SMA10',
       'SMA15', 'EMA5', 'EMA10', 'EMA20', 'EMA50', 'MACD', 'MACD_Signal',
       'MACD_Histogram'],
      dtype='object')

In [33]:
"""BOLLINGER BANDS (BB)"""
# BB 1 of 2 -> standard bollinger bands (20D CLOSE $)
# Set the number of periods for calculating SMA and the number of standard deviations
period = 20
std_dev_multiplier = 2

# Calculate the SMA for the middle band and name it 'middle_band_SMA20'
chart_historical_df[f'middle_band_SMA{period}'] = chart_historical_df['close'].rolling(window=period).mean()

# Calculate the standard deviation
chart_historical_df[f'std_dev_SMA{period}'] = chart_historical_df['close'].rolling(window=period).std()

# Calculate the upper and lower bands and name them 'upper_band_SMA20' and 'lower_band_SMA20'
chart_historical_df[f'upper_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] + (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)
chart_historical_df[f'lower_band_SMA{period}'] = chart_historical_df[f'middle_band_SMA{period}'] - (chart_historical_df[f'std_dev_SMA{period}'] * std_dev_multiplier)


# BB 2 of 2 -> typical price bollinger bands (20D TYPICAL $)




# view df with BBs (2) added
chart_historical_df.iloc[:-1]


In [54]:
# view df with BBs (2) added
# view the last row and the first row of the dataframe using slicing and iloc
chart_historical_df.iloc[-1]




symbol                      PBM
close                      1.27
high                        1.5
low                      0.9401
open                     0.9788
volume                  3711232
SMA5                      59.06
SMA10                    55.005
SMA15                   47.9547
EMA5                   45.77481
EMA10                 49.331257
EMA20                 49.543596
EMA50                 49.307166
MACD                   0.085664
MACD_Signal            1.019144
MACD_Histogram         -0.93348
middle_band_SMA20     49.309125
std_dev_SMA20         37.028541
upper_band_SMA20     123.366206
lower_band_SMA20     -24.747956
RSI                   42.792503
ATR                     49.6103
Name: 2024-03-07 00:00:00, dtype: object

| CALCULATIONS - J. Welles Wilder Jr (technical indicators) |
|-|
| *displayed underneath the chart not overlayed* |
| START |

In [34]:
"""RELATIVE STRENGTH INDEX (RSI)"""
def calculate_rsi(dataframe, periods=14):
    close_delta = dataframe['close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)

    ma_up = up.rolling(window=periods).mean()
    ma_down = down.rolling(window=periods).mean()

    rsi = ma_up / (ma_up + ma_down) * 100
    return rsi

# Adding the RSI to your DataFrame
chart_historical_df['RSI'] = calculate_rsi(chart_historical_df)

chart_historical_df.tail(10)


,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,RSI
date,,,,,,,,,,,,,,,,,,,,,
2024-03-05,CRMT,62.55,64.570,62.4600,63.5200,160505,26.5461,49.81105,46.582833,37.995806,41.506198,44.914993,47.436577,-3.278784,-1.185675,-2.093109,46.540625,35.611710,117.764045,-24.682795,47.189566
2024-03-05,HCI,97.08,100.050,96.8200,98.7200,92858,38.7821,56.02005,46.546833,57.690537,51.610526,49.883089,49.383378,1.314903,-0.685559,2.000462,49.705625,37.195596,124.096816,-24.685566,58.597487
2024-03-06,IOT,33.35,34.300,33.1600,33.6100,3810553,45.3280,49.41205,48.733500,49.577025,48.290430,48.308509,48.754618,-0.184915,-0.585431,0.400516,46.491625,35.578600,117.648826,-24.665576,46.956074
2024-03-06,PBM,1.02,1.380,0.7500,0.7900,6566906,38.9720,42.74505,44.312167,33.391350,39.695807,43.804841,46.882672,-3.936909,-1.255726,-2.681183,46.514125,35.548160,117.610445,-24.582195,46.994793
2024-03-06,CRMT,60.75,63.130,59.2900,61.0000,211956,50.9500,42.40205,46.058833,42.510900,43.523842,45.418666,47.426489,-2.066855,-1.417952,-0.648903,46.207625,35.389394,116.986413,-24.571163,55.155607
2024-03-06,HCI,98.16,98.400,97.0250,97.6100,115636,58.0720,42.30905,52.564700,61.060600,53.457689,50.441650,49.416039,2.406115,-0.653139,3.059253,49.455125,37.073867,123.602859,-24.692609,55.384336
2024-03-07,CRMT,62.30,63.735,60.6900,61.9600,194314,51.1160,44.94905,54.385367,61.473733,55.065382,51.571017,49.921292,3.022531,0.081995,2.940536,47.689125,35.465121,118.619367,-23.241117,46.673112
2024-03-07,IOT,34.36,34.400,33.0500,34.0000,7593193,51.3180,48.32300,50.047367,52.435822,51.300767,49.931872,49.311045,1.242202,0.314037,0.928165,49.379625,33.869823,117.119272,-18.360022,46.993120
2024-03-07,HCI,99.21,99.960,97.1050,98.5100,96791,70.9560,54.96400,52.148700,68.027215,60.011536,54.625027,51.267867,5.006421,1.252513,3.753907,50.973125,35.471145,121.915414,-19.969164,52.845335


In [35]:
"""AVERAGE TRUE RANGE (ATR)"""
def calculate_atr(dataframe, period=14):
    high_low = dataframe['high'] - dataframe['low']
    high_close = (dataframe['high'] - dataframe['close'].shift()).abs()
    low_close = (dataframe['low'] - dataframe['close'].shift()).abs()

    # Calculate the True Range (TR) as the maximum of the three measures
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # Calculate the ATR by taking the rolling mean of the TR
    atr = tr.rolling(window=period).mean()
    
    return atr

# Adding the ATR to your DataFrame
chart_historical_df['ATR'] = calculate_atr(chart_historical_df)

chart_historical_df.tail(10)

,symbol,close,high,low,open,volume,SMA5,SMA10,SMA15,EMA5,EMA10,EMA20,EMA50,MACD,MACD_Signal,MACD_Histogram,middle_band_SMA20,std_dev_SMA20,upper_band_SMA20,lower_band_SMA20,RSI,ATR
date,,,,,,,,,,,,,,,,,,,,,,
2024-03-05,CRMT,62.55,64.570,62.4600,63.5200,160505,26.5461,49.81105,46.582833,37.995806,41.506198,44.914993,47.436577,-3.278784,-1.185675,-2.093109,46.540625,35.611710,117.764045,-24.682795,47.189566,45.731279
2024-03-05,HCI,97.08,100.050,96.8200,98.7200,92858,38.7821,56.02005,46.546833,57.690537,51.610526,49.883089,49.383378,1.314903,-0.685559,2.000462,49.705625,37.195596,124.096816,-24.685566,58.597487,41.475636
2024-03-06,IOT,33.35,34.300,33.1600,33.6100,3810553,45.3280,49.41205,48.733500,49.577025,48.290430,48.308509,48.754618,-0.184915,-0.585431,0.400516,46.491625,35.578600,117.648826,-24.665576,46.956074,41.135636
2024-03-06,PBM,1.02,1.380,0.7500,0.7900,6566906,38.9720,42.74505,44.312167,33.391350,39.695807,43.804841,46.882672,-3.936909,-1.255726,-2.681183,46.514125,35.548160,117.610445,-24.582195,46.994793,41.043514
2024-03-06,CRMT,60.75,63.130,59.2900,61.0000,211956,50.9500,42.40205,46.058833,42.510900,43.523842,45.418666,47.426489,-2.066855,-1.417952,-0.648903,46.207625,35.389394,116.986413,-24.571163,55.155607,43.050157
2024-03-06,HCI,98.16,98.400,97.0250,97.6100,115636,58.0720,42.30905,52.564700,61.060600,53.457689,50.441650,49.416039,2.406115,-0.653139,3.059253,49.455125,37.073867,123.602859,-24.692609,55.384336,43.263157
2024-03-07,CRMT,62.30,63.735,60.6900,61.9600,194314,51.1160,44.94905,54.385367,61.473733,55.065382,51.571017,49.921292,3.022531,0.081995,2.940536,47.689125,35.465121,118.619367,-23.241117,46.673112,41.227450
2024-03-07,IOT,34.36,34.400,33.0500,34.0000,7593193,51.3180,48.32300,50.047367,52.435822,51.300767,49.931872,49.311045,1.242202,0.314037,0.928165,49.379625,33.869823,117.119272,-18.360022,46.993120,40.948164
2024-03-07,HCI,99.21,99.960,97.1050,98.5100,96791,70.9560,54.96400,52.148700,68.027215,60.011536,54.625027,51.267867,5.006421,1.252513,3.753907,50.973125,35.471145,121.915414,-19.969164,52.845335,45.229593


In [36]:
# START HERE
# see about if i should be doing a multi-index and group by if it makes the code more efficient and accurate and all that
# simple data plot soon to see what we are working with and if there are bugs

In [37]:
"""Average Directional Index (ADX)"""

'Average Directional Index (ADX)'

In [38]:
"""PARABOLIC SAR (PSAR)"""

'PARABOLIC SAR (PSAR)'

| J. Welles Wilder Jr (technical indicators) |
|-|
| END |

In [39]:
# LINE LEVELS not full calculation dataframe
# part 1 was calculations append
# part 2 is data points append
# part 3 extra - could be plotting past signals (might integrate ai in here like tensorflow)

| CHARTS |
|-|

# DELL FOCUS - Pre-market

In [40]:
# # $DELL
# dell_df = key_levels_df[key_levels_df['symbol'] == 'DELL'].copy()
# dell_df['date'] = pd.to_datetime(dell_df['date'])
# dell_df.set_index('date', inplace=True)
# dell_df.head(5)

In [41]:
# import mplfinance as mpf
# # candlestick chart
# # MANDATORY DATA = 'open', 'high', 'low', 'close', 'volume'

# """
# Part 1:    
# """
# # Part 1a: SMAs -> calculate
# dell_df['SMA20'] = dell_df['close'].rolling(window=20, min_periods=1).mean()
# dell_df['SMA50'] = dell_df['close'].rolling(window=50, min_periods=1).mean()
# # dell_df['SMA200'] = dell_df['close'].rolling(window=200, min_periods=1).mean()

# # Part 1b: VWAP -> calculate
# dell_df['VWAP'] = (dell_df['volume'] * (dell_df['high'] + dell_df['low'] + dell_df['close']) / 3).cumsum() / dell_df['volume'].cumsum()

# """   
# Part 2: 
# """
# add2plot = [
#     #### SMAs ####
#     mpf.make_addplot(dell_df['SMA20'], color='#7BAFD4', width=1.0, label='SMA 20'), # UNC blue
#     mpf.make_addplot(dell_df['SMA50'], color='#003087', width=1.0, label='SMA 50'), # DUKE blue
#     # mpf.make_addplot(dell_df['SMA200'], color='#041E42', width=1.0, label='SMA 200'), # HOYAS blue
#     #### VWAP ####
#     mpf.make_addplot(dell_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP') # BIG RED
# ]

# """   
# Part 3: styling not iterative changes related to data (i.e., color, font, etc.)
# """
# # Use a dark theme style and set text color to white
# mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})

# # Plot the candlestick chart with the moving averages
# fig, axes = mpf.plot(dell_df,
#                      type='candle',
#                      addplot=add2plot,
#                      volume=True,
#                     #  style=mpf_style,
#                      style=s, 
#                      # WTF look at docs (it's not erroring out but going to default style just figure out what that is i think)
#                      figratio=(15, 8),
#                      title='$DELL - Historical Data (6 Months)',
#                      tight_layout=True,
#                      returnfig=True)

# # Add a watermark if needed
# fig.text(0.5, 0.5, 'DELL', fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

# plt.show()


In [42]:
# dell_df.head(5)

In [43]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [44]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [45]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [46]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [47]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [48]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [49]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [50]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)